In [ ]:
# Cell 1: Install Dependencies
!pip install flask flask-cors pyngrok diffusers[torch] transformers accelerate controlnet_aux opencv-python-headless -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 9.6 MB/s eta 0:00:00


In [ ]:
# Cell 2: API Server Code

# --- Standard Imports ---
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import numpy as np
import cv2
from PIL import Image
import base64
import io
import os

# --- Ngrok Import for Public URL ---
from pyngrok import ngrok

# ❗️ Paste your authtoken here
ngrok.set_auth_token("2zyx1ApkLVnRdtWB5bZ1881hKon_88ag7e3Y9YYjb6mmyW9DR")

# --- 1. Initialize Flask App and CORS ---
app = Flask(__name__)
CORS(app)

# --- 2. Load AI Models ---
print("Loading models... This may take a few minutes the first time.")
# Set a cache directory inside Colab's writable space
cache_dir = "/content/.cache"
os.makedirs(cache_dir, exist_ok=True)

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16,
    cache_dir=cache_dir
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    cache_dir=cache_dir
).to("cuda") # Use "cuda" since Colab has a GPU

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload() # Helps manage memory
print("Models loaded successfully.")


# --- 3. Define the API Endpoint ---
@app.route('/generate', methods=['POST'], strict_slashes=False)
def generate():
    try:
        image_file = request.files['image']
        prompt = request.form['prompt']
        input_image = Image.open(io.BytesIO(image_file.read())).convert("RGB")

        # Canny Edge Detection
        np_image = np.array(input_image)
        canny_image = cv2.Canny(np_image, 100, 200)
        canny_image = canny_image[:, :, None]
        canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
        canny_image = Image.fromarray(canny_image)

        # Run the AI Pipeline
        output_image = pipe(
            prompt, image=canny_image, negative_prompt="low quality, blurry, ugly", num_inference_steps=20
        ).images[0]

        # Convert output image to Base64
        buffered = io.BytesIO()
        output_image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return jsonify({'image_base64': img_str})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# --- 4. Start the Server with ngrok ---
# This will create a public URL and start the Flask app
port = 7860
public_url = ngrok.connect(port)
print(f"✅ Public Tunnel URL: {public_url}")
print("Server is running. Do not stop this cell.")
app.run(port=port)

Loading models... This may take a few minutes the first time.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Models loaded successfully.
✅ Public Tunnel URL: NgrokTunnel: "https://1aba253ed52f.ngrok-free.app" -> "http://localhost:7860"
Server is running. Do not stop this cell.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:7860
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Sep/2025 12:53:39] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Sep/2025 12:53:40] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# ============================================================
# Cell 1: Install Dependencies
# ============================================================
!pip install flask flask-cors pyngrok diffusers[torch] transformers accelerate controlnet_aux opencv-python-headless -q

# ============================================================
# Cell 2: API Server Code
# ============================================================
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import numpy as np
import cv2
from PIL import Image
import base64
import io
import os

# --- Ngrok Import for Public URL ---
from pyngrok import ngrok

# ⚠️ Paste your ngrok authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2zyx1ApkLVnRdtWB5bZ1881hKon_88ag7e3Y9YYjb6mmyW9DR")

# --- 1. Initialize Flask App and CORS ---
app = Flask(__name__)
CORS(app)

# --- 2. Load AI Models ---
print("Loading models... This may take a few minutes the first time.")
cache_dir = "/content/.cache"
os.makedirs(cache_dir, exist_ok=True)

# ✅ Use torch_dtype instead of dtype
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16,
    cache_dir=cache_dir
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    cache_dir=cache_dir
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
print("✅ Models loaded successfully.")

# ============================================================
# Cell 3: Define Flask Routes
# ============================================================

# Home page with HTML form
@app.route("/", methods=["GET"])
def home():
    return """
    <h2>🚀 Crafter's Corner AI Image Generator</h2>
    <p>Upload a sketch/photo and enter a text prompt to generate a colorful AI image.</p>

    <form action="/generate" method="post" enctype="multipart/form-data">
        <label for="prompt">Prompt:</label><br>
        <input type="text" id="prompt" name="prompt" style="width:300px" required><br><br>

        <label for="image">Upload Image:</label><br>
        <input type="file" id="image" name="image" accept="image/*" required><br><br>

        <button type="submit">Generate</button>
    </form>
    """

@app.route('/generate', methods=['POST'], strict_slashes=False)
def generate():
    try:
        image_file = request.files['image']
        prompt = request.form['prompt']
        input_image = Image.open(io.BytesIO(image_file.read())).convert("RGB")

        # Canny Edge Detection
        np_image = np.array(input_image)
        canny_image = cv2.Canny(np_image, 100, 200)
        canny_image = canny_image[:, :, None]
        canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)
        canny_image = Image.fromarray(canny_image)

        # Run the AI Pipeline
        output_image = pipe(
            prompt,
            image=canny_image,
            negative_prompt="low quality, blurry, ugly",
            num_inference_steps=20
        ).images[0]

        # Convert output image to Base64
        buffered = io.BytesIO()
        output_image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        # If browser form → return HTML page
        if "text/html" in request.headers.get("Accept", "") or "curl" not in request.headers.get("User-Agent", "").lower():
            return f"""
            <h2>🖼️ Generated Image</h2>
            <p><b>Prompt:</b> {prompt}</p>
            <img src="data:image/png;base64,{img_str}" style="max-width:500px; border:1px solid #ccc"/>
            <br><br>
            <a href="/">🔙 Generate another</a>
            """
        else:
            # From code (axios/curl) → return JSON
            return jsonify({'image_base64': img_str})

    except Exception as e:
        return f"<p style='color:red'>Error: {str(e)}</p>"

# ============================================================
# Cell 4: Start Server
# ============================================================
port = 7860
public_url = ngrok.connect(port)
print(f"✅ Public Tunnel URL: {public_url}")
print("Server is running. Do not stop this cell.")
app.run(port=port)


Loading models... This may take a few minutes the first time.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Models loaded successfully.
✅ Public Tunnel URL: NgrokTunnel: "https://32aac18136e1.ngrok-free.app" -> "http://localhost:7860"
Server is running. Do not stop this cell.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:7860
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Sep/2025 13:14:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Sep/2025 13:14:52] "GET /favicon.ico HTTP/1.1" 404 -


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [05/Sep/2025 13:15:45] "POST /generate HTTP/1.1" 200 -


In [1]:
# ============================================================
# Cell 1: Install Dependencies
# ============================================================
!pip install flask flask-cors pyngrok "diffusers[torch]" transformers accelerate controlnet_aux opencv-python-headless -q

# ============================================================
# Cell 2: API Server Code
# ============================================================
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from diffusers import StableDiffusionPipeline, StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import numpy as np
import cv2
from PIL import Image
import base64, io, os
from pyngrok import ngrok

# --- 1. Setup Flask + CORS ---
app = Flask(__name__)
CORS(app)

# --- 2. Setup ngrok ---
ngrok.set_auth_token("2zyx1ApkLVnRdtWB5bZ1881hKon_88ag7e3Y9YYjb6mmyW9DR")  # replace with your ngrok token

# --- 3. Load Models ---
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print(f"✅ Using device: {device}, dtype: {dtype}")
cache_dir = "/content/.cache"
os.makedirs(cache_dir, exist_ok=True)

# Text-to-image pipeline
base_pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype,
    cache_dir=cache_dir
).to(device)
base_pipe.scheduler = UniPCMultistepScheduler.from_config(base_pipe.scheduler.config)

# ControlNet for sketches
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=dtype,
    cache_dir=cache_dir
)
controlnet_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=dtype,
    cache_dir=cache_dir
).to(device)
controlnet_pipe.scheduler = UniPCMultistepScheduler.from_config(controlnet_pipe.scheduler.config)

print("✅ Models loaded successfully.")

# --- Helper: Encode image as base64 ---
def pil_to_base64(img: Image.Image) -> str:
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode("utf-8")

# --- Routes ---
@app.route("/", methods=["GET"])
def home():
    return {"status": "running"}

@app.route("/generate", methods=["POST"])
def generate():
    try:
        prompt = request.form["prompt"]
        image_file = request.files.get("image", None)

        if image_file:  # sketch-to-image
            input_image = Image.open(io.BytesIO(image_file.read())).convert("RGB")
            input_image = input_image.resize((256, 256))

            # Apply Canny
            np_image = np.array(input_image)
            canny_edges = cv2.Canny(np_image, 100, 200)
            canny_edges = np.repeat(canny_edges[:, :, None], 3, axis=2)
            canny_image = Image.fromarray(canny_edges)

            output_image = controlnet_pipe(
                prompt,
                image=canny_image,
                negative_prompt="low quality, blurry, ugly",
                num_inference_steps=15
            ).images[0]
        else:  # text-to-image
            output_image = base_pipe(
                prompt,
                negative_prompt="low quality, blurry, ugly",
                num_inference_steps=20
            ).images[0]

        return jsonify({"image_base64": pil_to_base64(output_image)})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# --- Start server with ngrok ---
port = 7860
public_url = ngrok.connect(port)
print(f"🌍 Public backend URL: {public_url}")
app.run(port=port)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 12.6 MB/s eta 0:00:00
✅ Using device: cuda, dtype: torch.float16


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
INFO:werkzeug:127.0.0.1 - - [08/Sep/2025 07:57:03] "POST /generate HTTP/1.1" 200 -


  0%|          | 0/15 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [08/Sep/2025 07:58:18] "POST /generate HTTP/1.1" 200 -


  0%|          | 0/15 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [08/Sep/2025 07:58:38] "POST /generate HTTP/1.1" 200 -
